# 🤖 Week 16 - Day 1: Introduction to AI Agents

## Today's Goals:
✅ Understand what AI Agents are and why they matter

✅ Learn LangGraph fundamentals (Nodes, Edges, State)

✅ Create custom tools using the `@tool` decorator

✅ Build your first AI Agent with LangGraph

✅ Test agents with multiple tools

---

## 🔧 Part 1: Setup - Install & Import All Libraries

**IMPORTANT:** Run ALL cells in this part sequentially!

In [2]:
# STEP 1: Install required packages
print("📦 Installing packages... (this may take 1-2 minutes)\n")

# IMPORTANT: Install with compatible versions to avoid conflicts
!pip install -q langchain-core==0.3.29 langchain==0.3.14 langchain-community==0.3.14
!pip install -q langgraph==0.2.61 langchain-groq==0.2.3
!pip install -q python-dotenv

print("\n✅ All packages installed successfully!")
print("\n⚠️ NOTE: If you see version warnings above, they can be ignored.")
print("The packages will still work correctly for this notebook.")

📦 Installing packages... (this may take 1-2 minutes)



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.29 which is incompatible.
langchain-openai 0.1.7 requires langchain-core<0.3,>=0.1.46, but you have langchain-core 0.3.29 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.29 which is incompatible.



✅ All packages installed successfully!

⚠️ NOTE: If you see version warnings above, they can be ignored.
The packages will still work correctly for this notebook.


In [3]:
# STEP 2: Import ALL libraries
import os
import warnings
warnings.filterwarnings('ignore')

# LangGraph Core
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent, ToolNode
from langgraph.graph.message import add_messages

# LangChain Core
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# LLM
from langchain_groq import ChatGroq

# Type hints
from typing import Annotated, TypedDict, Literal

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [4]:
# STEP 3: Set up API Key for Groq (Free LLM API)
# Get your free API key from: https://console.groq.com/

# Option 1: Set directly (for learning - don't do this in production!)
GROQ_API_KEY = "your-groq-api-key-here"  # Replace with your actual key

# Option 2: Use environment variable (recommended)
# os.environ["GROQ_API_KEY"] = "your-key-here"

os.environ["GROQ_API_KEY"] = GROQ_API_KEY

print("✅ API key configured!")
print("\n💡 Get a FREE Groq API key at: https://console.groq.com/")
print("🚀 Ready to build AI Agents!")

✅ API key configured!

💡 Get a FREE Groq API key at: https://console.groq.com/
🚀 Ready to build AI Agents!


---

## 🧠 Part 2: Understanding AI Agents

Before we code, let's understand what we're building!

### 🎯 What is an AI Agent?

**An AI Agent is an LLM that can:**
1. **Reason** about what to do next
2. **Take actions** using tools
3. **Observe** the results
4. **Decide** if more actions are needed

### 🔄 The Agent Loop (ReAct Pattern):

```
User Query → 🤔 REASON → 🔧 ACT → 👀 OBSERVE → 🤔 REASON → ... → ✅ Final Answer
```

### 💡 Key Difference: Chain vs Agent

| Chain | Agent |
|-------|-------|
| Fixed steps | Dynamic decisions |
| Always same path | Chooses which tool |
| No loops | Can retry/iterate |
| Predictable | Autonomous |

---

## 🔷 Part 3: LangGraph Fundamentals

LangGraph helps us build agents using **graphs**. Let's understand the core concepts:

### 📊 Graph Components:

```
┌─────────────────────────────────────────────────────┐
│                    LangGraph                        │
├─────────────────────────────────────────────────────┤
│                                                     │
│   📍 NODES = Units of work (functions)              │
│      • LLM calls                                    │
│      • Tool execution                               │
│      • Data processing                              │
│                                                     │
│   ➡️ EDGES = Connections between nodes              │
│      • Normal: A → B (always)                       │
│      • Conditional: A → B or C (based on logic)     │
│                                                     │
│   📦 STATE = Shared data across nodes               │
│      • Messages history                             │
│      • Variables                                    │
│      • Context                                      │
│                                                     │
└─────────────────────────────────────────────────────┘
```

In [5]:
# Let's understand State first - it's the "memory" of our agent

# Define a simple State using TypedDict
class SimpleState(TypedDict):
    """State that holds messages between nodes"""
    messages: Annotated[list, add_messages]  # List of messages with auto-append

print("📦 State Definition:")
print("   - messages: A list that stores all conversation messages")
print("   - add_messages: Automatically appends new messages (doesn't replace)")
print("\n💡 Think of State as a shared notebook that every node can read and write to!")

📦 State Definition:
   - messages: A list that stores all conversation messages
   - add_messages: Automatically appends new messages (doesn't replace)

💡 Think of State as a shared notebook that every node can read and write to!


In [6]:
# Let's build a simple graph to understand the flow

# Step 1: Define a simple node function
def greet_node(state: SimpleState):
    """A simple node that adds a greeting message"""
    return {"messages": [AIMessage(content="Hello! I'm your AI assistant. How can I help?")]}

def process_node(state: SimpleState):
    """A node that processes the last message"""
    last_message = state["messages"][-1].content
    response = f"I received your message: '{last_message}'. Processing complete!"
    return {"messages": [AIMessage(content=response)]}

print("✅ Node functions defined!")
print("\n📍 Nodes are just Python functions that:")
print("   1. Take state as input")
print("   2. Do some work")
print("   3. Return updated state")

✅ Node functions defined!

📍 Nodes are just Python functions that:
   1. Take state as input
   2. Do some work
   3. Return updated state


In [7]:
# Step 2: Build the graph

# Create a new graph with our state
simple_graph = StateGraph(SimpleState)

# Add nodes
simple_graph.add_node("greeter", greet_node)
simple_graph.add_node("processor", process_node)

# Add edges (connections)
simple_graph.add_edge(START, "greeter")      # Start → Greeter
simple_graph.add_edge("greeter", "processor") # Greeter → Processor
simple_graph.add_edge("processor", END)       # Processor → End

# Compile the graph
simple_app = simple_graph.compile()

print("✅ Graph built and compiled!")
print("\n📊 Graph Structure:")
print("   START → greeter → processor → END")

✅ Graph built and compiled!

📊 Graph Structure:
   START → greeter → processor → END


In [8]:
# Step 3: Run the graph!

# Initial state with a user message
initial_state = {
    "messages": [HumanMessage(content="What can you do?")]
}

# Run the graph
result = simple_app.invoke(initial_state)

print("🚀 Graph Execution Results:\n")
print("=" * 50)
for i, msg in enumerate(result["messages"]):
    role = "Human" if isinstance(msg, HumanMessage) else "AI"
    print(f"\n{role}: {msg.content}")
print("\n" + "=" * 50)

🚀 Graph Execution Results:


Human: What can you do?

AI: Hello! I'm your AI assistant. How can I help?

AI: I received your message: 'Hello! I'm your AI assistant. How can I help?'. Processing complete!



### 💡 Key Insights:

✅ **StateGraph** creates a workflow from nodes and edges

✅ **Nodes** are functions that process and update state

✅ **Edges** define the flow between nodes

✅ **compile()** turns the graph into a runnable application

---

## 🛠️ Part 4: Creating Tools with @tool Decorator

Tools give agents **superpowers** - the ability to take real actions!

### What makes a good tool?
1. **Clear name** - describes what it does
2. **Docstring** - helps LLM understand when to use it
3. **Type hints** - define input/output types
4. **Single purpose** - does one thing well

In [9]:
# Let's create some simple tools!

@tool
def add(a: int, b: int) -> int:
    """Add two numbers together.
    
    Args:
        a: First number
        b: Second number
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers together.
    
    Args:
        a: First number
        b: Second number
    """
    return a * b

@tool
def subtract(a: int, b: int) -> int:
    """Subtract the second number from the first.
    
    Args:
        a: First number
        b: Second number to subtract
    """
    return a - b

print("✅ Calculator tools created!")
print("\n🔧 Available tools:")
print(f"   1. {add.name}: {add.description}")
print(f"   2. {multiply.name}: {multiply.description}")
print(f"   3. {subtract.name}: {subtract.description}")

✅ Calculator tools created!

🔧 Available tools:
   1. add: Add two numbers together.

    Args:
        a: First number
        b: Second number
   2. multiply: Multiply two numbers together.

    Args:
        a: First number
        b: Second number
   3. subtract: Subtract the second number from the first.

    Args:
        a: First number
        b: Second number to subtract


In [10]:
# Let's examine what the @tool decorator creates

print("🔍 Examining the 'add' tool:\n")
print(f"📛 Name: {add.name}")
print(f"📝 Description: {add.description}")
print(f"📊 Arguments: {add.args}")
print(f"\n💡 The LLM uses this information to decide WHEN and HOW to call the tool!")

🔍 Examining the 'add' tool:

📛 Name: add
📝 Description: Add two numbers together.

    Args:
        a: First number
        b: Second number
📊 Arguments: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}

💡 The LLM uses this information to decide WHEN and HOW to call the tool!


In [11]:
# Test the tools manually

print("🧪 Testing tools manually:\n")

# Tools can be invoked directly
result1 = add.invoke({"a": 5, "b": 3})
print(f"add(5, 3) = {result1}")

result2 = multiply.invoke({"a": 4, "b": 7})
print(f"multiply(4, 7) = {result2}")

result3 = subtract.invoke({"a": 10, "b": 4})
print(f"subtract(10, 4) = {result3}")

print("\n✅ All tools working correctly!")

🧪 Testing tools manually:

add(5, 3) = 8
multiply(4, 7) = 28
subtract(10, 4) = 6

✅ All tools working correctly!


In [12]:
# Let's create more interesting tools!

@tool
def get_word_length(word: str) -> int:
    """Get the length of a word (number of characters).
    
    Args:
        word: The word to measure
    """
    return len(word)

@tool
def reverse_string(text: str) -> str:
    """Reverse a string (spell it backwards).
    
    Args:
        text: The text to reverse
    """
    return text[::-1]

@tool
def count_vowels(text: str) -> int:
    """Count the number of vowels (a, e, i, o, u) in a text.
    
    Args:
        text: The text to analyze
    """
    vowels = "aeiouAEIOU"
    return sum(1 for char in text if char in vowels)

print("✅ String tools created!")
print("\n🔧 New tools available:")
print(f"   • get_word_length: {get_word_length.description}")
print(f"   • reverse_string: {reverse_string.description}")
print(f"   • count_vowels: {count_vowels.description}")

✅ String tools created!

🔧 New tools available:
   • get_word_length: Get the length of a word (number of characters).

    Args:
        word: The word to measure
   • reverse_string: Reverse a string (spell it backwards).

    Args:
        text: The text to reverse
   • count_vowels: Count the number of vowels (a, e, i, o, u) in a text.

    Args:
        text: The text to analyze


### 💡 Key Insights:

✅ **@tool decorator** converts functions into LangChain tools

✅ **Docstrings are CRITICAL** - they help the LLM understand when to use the tool

✅ **Type hints are REQUIRED** - they define the input schema

✅ **Tools can be tested** directly with `.invoke()`

---

## 🤖 Part 5: Building Your First AI Agent

Now let's combine everything to build a real AI Agent!

We'll use LangGraph's `create_react_agent` - a prebuilt agent that implements the ReAct pattern.

In [13]:
# Step 1: Initialize the LLM

llm = ChatGroq(
    model="llama-3.1-8b-instant",  # Fast and capable!
    temperature=0,  # Deterministic outputs
    max_tokens=500
)

print("✅ LLM initialized!")
print("📊 Model: Llama 3.1 8B (via Groq)")
print("⚡ Groq provides ultra-fast inference!")

✅ LLM initialized!
📊 Model: Llama 3.1 8B (via Groq)
⚡ Groq provides ultra-fast inference!


In [14]:
# Step 2: Create the agent with tools

# Collect our math tools
math_tools = [add, multiply, subtract]

# Create the agent using create_react_agent (easiest way!)
math_agent = create_react_agent(
    model=llm,
    tools=math_tools
)

print("✅ Math Agent created!")
print("\n🤖 Agent capabilities:")
print("   • Can add numbers")
print("   • Can multiply numbers")
print("   • Can subtract numbers")
print("   • Decides which tool to use based on the question!")

✅ Math Agent created!

🤖 Agent capabilities:
   • Can add numbers
   • Can multiply numbers
   • Can subtract numbers
   • Decides which tool to use based on the question!


In [15]:
# Step 3: Test the agent!

def ask_agent(agent, question):
    """Helper function to ask questions to an agent"""
    print(f"\n❓ Question: {question}")
    print("=" * 50)
    
    # Run the agent
    result = agent.invoke({
        "messages": [HumanMessage(content=question)]
    })
    
    # Display all messages
    print("\n📝 Agent's Thought Process:")
    for msg in result["messages"]:
        if isinstance(msg, HumanMessage):
            print(f"\n👤 Human: {msg.content}")
        elif isinstance(msg, AIMessage):
            if msg.tool_calls:
                for tc in msg.tool_calls:
                    print(f"\n🔧 Tool Call: {tc['name']}({tc['args']})")
            else:
                print(f"\n🤖 AI: {msg.content}")
        else:
            # Tool message
            print(f"\n📤 Tool Result: {msg.content}")
    
    print("\n" + "=" * 50)
    return result

In [16]:
# Test 1: Simple addition
result1 = ask_agent(math_agent, "What is 15 plus 27?")


❓ Question: What is 15 plus 27?


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [ ]:
# Test 2: Multiplication
result2 = ask_agent(math_agent, "Can you multiply 8 and 9?")

In [ ]:
# Test 3: Multi-step calculation
result3 = ask_agent(math_agent, "What is 10 plus 5, then multiply that by 3?")

In [ ]:
# Test 4: Agent decides which tool to use
result4 = ask_agent(math_agent, "I have 100 apples and give away 37. How many do I have left?")

### 💡 Key Observations:

✅ **The agent REASONS** about which tool to use

✅ **It ACTS** by calling the appropriate tool

✅ **It OBSERVES** the result

✅ **It DECIDES** if more actions are needed or provides the final answer

This is the **ReAct pattern** in action!

---

## 🚀 Part 6: Agent with Multiple Tool Types

Let's create a more versatile agent with different types of tools!

In [ ]:
# Create an agent with both math and string tools

all_tools = [
    add, 
    multiply, 
    subtract,
    get_word_length,
    reverse_string,
    count_vowels
]

# Create a versatile agent
versatile_agent = create_react_agent(
    model=llm,
    tools=all_tools
)

print("✅ Versatile Agent created!")
print("\n🤖 This agent can:")
print("   📊 Math: add, multiply, subtract")
print("   📝 Text: word length, reverse, count vowels")

In [ ]:
# Test with string operations
result5 = ask_agent(versatile_agent, "How many characters are in the word 'LangGraph'?")

In [ ]:
# Test string reversal
result6 = ask_agent(versatile_agent, "What is 'artificial' spelled backwards?")

In [ ]:
# Test vowel counting
result7 = ask_agent(versatile_agent, "How many vowels are in the word 'intelligence'?")

In [ ]:
# Test mixed: Agent chooses the right tool!
result8 = ask_agent(versatile_agent, "Add 5 and 3, then tell me how many vowels are in the word 'eight'")

---

## 🔨 Part 7: Building an Agent from Scratch (Custom Graph)

Let's understand how agents work internally by building one step-by-step!

In [ ]:
# Step 1: Define the State for our agent
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """State for our custom agent"""
    messages: Annotated[list, add_messages]

print("✅ Agent State defined!")

In [ ]:
# Step 2: Create the LLM with tools bound to it

# Our tools for this agent
tools = [add, multiply, subtract]

# Bind tools to the LLM - this lets the LLM know about available tools
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM bound with tools!")
print("\n💡 bind_tools() tells the LLM:")
print("   • What tools are available")
print("   • How to call them (arguments)")
print("   • When to use them (based on descriptions)")

In [ ]:
# Step 3: Define the "agent" node - calls the LLM

def agent_node(state: AgentState):
    """The agent node: calls the LLM to decide what to do"""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

print("✅ Agent node defined!")

In [ ]:
# Step 4: Create the tool node using LangGraph's prebuilt ToolNode

tool_node = ToolNode(tools)

print("✅ Tool node created!")
print("\n💡 ToolNode automatically:")
print("   • Reads tool calls from the last message")
print("   • Executes the appropriate tool")
print("   • Returns the result as a message")

In [ ]:
# Step 5: Define the conditional edge - should we continue or stop?

def should_continue(state: AgentState) -> Literal["tools", "end"]:
    """Decide whether to use tools or end the conversation"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the LLM wants to call a tool, go to tools node
    if last_message.tool_calls:
        return "tools"
    
    # Otherwise, we're done
    return "end"

print("✅ Conditional edge function defined!")
print("\n💡 This function decides:")
print("   • If LLM made tool_calls → go to 'tools' node")
print("   • Otherwise → go to 'end' (finish)")

In [ ]:
# Step 6: Build the graph!

# Create the graph
custom_graph = StateGraph(AgentState)

# Add nodes
custom_graph.add_node("agent", agent_node)
custom_graph.add_node("tools", tool_node)

# Add edges
custom_graph.add_edge(START, "agent")  # Start → Agent

# Add conditional edge from agent
custom_graph.add_conditional_edges(
    "agent",  # From node
    should_continue,  # Function that decides
    {
        "tools": "tools",  # If should_continue returns "tools"
        "end": END  # If should_continue returns "end"
    }
)

# After tools, go back to agent (loop!)
custom_graph.add_edge("tools", "agent")

# Compile
custom_agent = custom_graph.compile()

print("✅ Custom Agent Graph compiled!")
print("\n📊 Graph Structure:")
print("   START → agent ⟷ tools")
print("              ↓")
print("             END")

In [ ]:
# Test our custom agent!
result9 = ask_agent(custom_agent, "What is 25 multiplied by 4?")

In [ ]:
# Test multi-step reasoning
result10 = ask_agent(custom_agent, "Calculate 50 minus 15, then add 10 to that result")

### 💡 Understanding the Agent Loop:

```
┌─────────────────────────────────────────────────────┐
│                                                     │
│   START                                             │
│     │                                               │
│     ▼                                               │
│  ┌──────┐    tool_calls?    ┌───────┐              │
│  │AGENT │ ───────Yes──────► │ TOOLS │              │
│  └──────┘                   └───────┘              │
│     │                           │                   │
│     │ No                        │                   │
│     │                           │                   │
│     ▼                           │                   │
│    END ◄────────────────────────┘                   │
│         (loop back to agent)                        │
│                                                     │
└─────────────────────────────────────────────────────┘
```

---

## 🎯 Part 8: Mini Challenge

### 🏆 Challenge: Create Your Own Agent!

**Your Tasks:**
1. Create 2-3 new tools (be creative!)
2. Build an agent with these tools
3. Test your agent with various questions

**Ideas for tools:**
- Temperature converter (Celsius ↔ Fahrenheit)
- BMI calculator
- Simple tip calculator
- Word counter
- Text to uppercase/lowercase

In [ ]:
# Your code here!
# Example structure:

# Step 1: Create your tools
# @tool
# def my_tool(param: type) -> return_type:
#     """Description of what this tool does"""
#     return result

# Step 2: Create your agent
# my_tools = [tool1, tool2, tool3]
# my_agent = create_react_agent(model=llm, tools=my_tools)

# Step 3: Test it!
# ask_agent(my_agent, "Your question here")

pass

In [ ]:
# Solution Example (Temperature Converter)

@tool
def celsius_to_fahrenheit(celsius: float) -> float:
    """Convert temperature from Celsius to Fahrenheit.
    
    Args:
        celsius: Temperature in Celsius
    """
    return (celsius * 9/5) + 32

@tool
def fahrenheit_to_celsius(fahrenheit: float) -> float:
    """Convert temperature from Fahrenheit to Celsius.
    
        Args:
        fahrenheit: Temperature in Fahrenheit
    """
    return (fahrenheit - 32) * 5/9

@tool
def calculate_tip(bill_amount: float, tip_percentage: float) -> float:
    """Calculate the tip amount for a bill.
    
    Args:
        bill_amount: The total bill amount
        tip_percentage: The tip percentage (e.g., 15 for 15%)
    """
    return bill_amount * (tip_percentage / 100)

# Create the agent
utility_tools = [celsius_to_fahrenheit, fahrenheit_to_celsius, calculate_tip]
utility_agent = create_react_agent(model=llm, tools=utility_tools)

print("✅ Utility Agent created!")

In [ ]:
# Test the utility agent
ask_agent(utility_agent, "What is 100 degrees Fahrenheit in Celsius?")

In [ ]:
# Test tip calculation
ask_agent(utility_agent, "My bill is $85. How much should I tip at 18%?")

---

## 📚 Summary - What We Learned Today

### 1. AI Agents Fundamentals 🤖
- Agents can **reason**, **act**, and **observe**
- They follow the **ReAct pattern** (Reasoning + Acting)
- Unlike chains, agents make **dynamic decisions**

### 2. LangGraph Core Concepts 🔷
- **State**: Shared data between nodes (like a notebook)
- **Nodes**: Units of work (Python functions)
- **Edges**: Connections that define flow
- **Conditional Edges**: Dynamic routing based on logic

### 3. Creating Tools 🛠️
- Use `@tool` decorator to create tools
- **Docstrings are essential** - they guide the LLM
- **Type hints are required** - they define the schema
- Tools can be tested with `.invoke()`

### 4. Building Agents 🚀
- `create_react_agent()` is the easiest way to start
- Custom graphs give more control
- The agent loop: Agent → Tools → Agent → ... → End

---

## 🎯 Key Takeaways

✅ **Agents = LLM + Tools + Decision Loop**

✅ **LangGraph makes it easy** to build agent workflows

✅ **Good tools have clear names and descriptions**

✅ **Start with `create_react_agent`**, then customize as needed

✅ **The agent decides** which tool to use and when

---

## 💡 Pro Tips

1. **Write clear docstrings** - The LLM reads them to understand tools
2. **Keep tools focused** - One tool, one job
3. **Test tools independently** before using in agents
4. **Use `create_react_agent`** for quick prototyping
5. **Build custom graphs** when you need more control

---

## 🚀 Next Steps - Tomorrow!

**Day 2: LangGraph Workflows**
- Conditional and parallel edges
- Multi-agent workflow creation
- Inter-agent communication
- Tool orchestration

**Get ready to build complex agent systems! 🚀**

---

## 🎉 Congratulations!

You've built your first AI Agents!

You now know how to:
- ✅ Understand what AI Agents are
- ✅ Work with LangGraph (State, Nodes, Edges)
- ✅ Create custom tools with `@tool`
- ✅ Build agents with `create_react_agent`
- ✅ Build custom agent graphs from scratch

**Keep practicing and see you tomorrow! 🚀**